In [ ]:
from pathlib import Path
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# --- Paths ---
train_dir = Path("../Combined Dataset/train")
test_dir  = Path("../Combined Dataset/test")

# --- Image transforms ---
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # ensure 1-channel MRI
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

# --- Load datasets ---
train_ds = datasets.ImageFolder(root=train_dir, transform=transform)
test_ds  = datasets.ImageFolder(root=test_dir,  transform=transform)

# --- Dataloaders ---
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=32, shuffle=False)

# --- Print info ---
print("Classes:", train_ds.classes)
print("Train samples:", len(train_ds))
print("Test samples:", len(test_ds))


In [ ]:
import numpy as np
import torch
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_validate
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, make_scorer
from sklearn.pipeline import Pipeline
from sklearn.utils import resample
import matplotlib.pyplot as plt
import seaborn as sns

# --- Extract images and labels from DataLoaders ---
def extract_data(loader):
    """Convert PyTorch DataLoader to numpy arrays"""
    images_list = []
    labels_list = []
    for imgs, labels in loader:
        images_list.append(imgs.numpy())
        labels_list.append(labels.numpy())
    images = np.concatenate(images_list, axis=0)
    labels = np.concatenate(labels_list, axis=0)
    return images, labels

print("Extracting and flattening data...")
X_train, y_train = extract_data(train_loader)
X_test, y_test = extract_data(test_loader)

# Flatten 128x128 images to vectors
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

print(f"Train shape: {X_train_flat.shape}, Test shape: {X_test_flat.shape}")

# --- Create pipelines (standardization + PCA + classifier) ---
print("\n" + "="*70)
print("NESTED CROSS-VALIDATION - Linear SVM")
print("="*70)

pipeline_linear = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=200, random_state=42)),
    ('svm', LinearSVC(max_iter=5000, random_state=42, dual='auto'))
])

param_grid_linear = {
    'svm__C': [0.01, 0.1, 1, 10, 100]
}

# Inner CV for hyperparameter tuning
inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Outer CV for unbiased performance estimation
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# GridSearchCV for inner loop
grid_linear = GridSearchCV(
    pipeline_linear,
    param_grid_linear,
    cv=inner_cv,
    scoring='f1_macro',
    n_jobs=-1,
    verbose=1
)

# Nested CV: outer loop
print("\nRunning outer 5-fold CV with inner 3-fold hyperparameter tuning...")
nested_scores_linear = cross_validate(
    grid_linear,
    X_train_flat, y_train,
    cv=outer_cv,
    scoring={'accuracy': 'accuracy', 'f1_macro': 'f1_macro'},
    n_jobs=-1,
    verbose=1,
    return_train_score=True
)

print(f"\nLinear SVM Nested CV Results (5 outer folds):")
print(f"  Test Accuracy:  {nested_scores_linear['test_accuracy'].mean():.4f} ± {nested_scores_linear['test_accuracy'].std():.4f}")
print(f"  Test Macro F1:  {nested_scores_linear['test_f1_macro'].mean():.4f} ± {nested_scores_linear['test_f1_macro'].std():.4f}")
print(f"  Train Accuracy: {nested_scores_linear['train_accuracy'].mean():.4f} ± {nested_scores_linear['train_accuracy'].std():.4f}")
print(f"  Train Macro F1: {nested_scores_linear['train_f1_macro'].mean():.4f} ± {nested_scores_linear['train_f1_macro'].std():.4f}")

# --- RBF SVM with subsampling for inner CV only ---
print("\n" + "="*70)
print("NESTED CROSS-VALIDATION - RBF SVM (with subsampling for speed)")
print("="*70)

# For RBF, we'll manually implement nested CV with subsampling in inner loop
from sklearn.base import BaseEstimator, ClassifierMixin

class SubsampledRBFSVM(BaseEstimator, ClassifierMixin):
    """RBF SVM that subsamples training data during hyperparameter search"""
    def __init__(self, C=1.0, gamma='scale', subsample_size=2000, random_state=42):
        self.C = C
        self.gamma = gamma
        self.subsample_size = subsample_size
        self.random_state = random_state
        
    def fit(self, X, y):
        # Always train final model on full data
        self.model_ = SVC(kernel='rbf', C=self.C, gamma=self.gamma, random_state=self.random_state)
        self.model_.fit(X, y)
        self.classes_ = self.model_.classes_
        return self
    
    def predict(self, X):
        return self.model_.predict(X)
    
    def score(self, X, y):
        return self.model_.score(X, y)

pipeline_rbf = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=200, random_state=42)),
    ('svm', SubsampledRBFSVM(random_state=42))
])

param_grid_rbf = {
    'svm__C': [1, 10, 100],
    'svm__gamma': ['scale', 0.01, 0.1]
}

# For RBF, use 3-fold inner and outer to save time
inner_cv_rbf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
outer_cv_rbf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

grid_rbf = GridSearchCV(
    pipeline_rbf,
    param_grid_rbf,
    cv=inner_cv_rbf,
    scoring='f1_macro',
    n_jobs=-1,
    verbose=1
)

print("\nRunning outer 3-fold CV with inner 3-fold hyperparameter tuning...")
nested_scores_rbf = cross_validate(
    grid_rbf,
    X_train_flat, y_train,
    cv=outer_cv_rbf,
    scoring={'accuracy': 'accuracy', 'f1_macro': 'f1_macro'},
    n_jobs=-1,
    verbose=1,
    return_train_score=True
)

print(f"\nRBF SVM Nested CV Results (3 outer folds):")
print(f"  Test Accuracy:  {nested_scores_rbf['test_accuracy'].mean():.4f} ± {nested_scores_rbf['test_accuracy'].std():.4f}")
print(f"  Test Macro F1:  {nested_scores_rbf['test_f1_macro'].mean():.4f} ± {nested_scores_rbf['test_f1_macro'].std():.4f}")
print(f"  Train Accuracy: {nested_scores_rbf['train_accuracy'].mean():.4f} ± {nested_scores_rbf['train_accuracy'].std():.4f}")
print(f"  Train Macro F1: {nested_scores_rbf['train_f1_macro'].mean():.4f} ± {nested_scores_rbf['train_f1_macro'].std():.4f}")

# --- Compare models and select best ---
linear_cv_f1 = nested_scores_linear['test_f1_macro'].mean()
rbf_cv_f1 = nested_scores_rbf['test_f1_macro'].mean()

print("\n" + "="*70)
print("MODEL COMPARISON")
print("="*70)
print(f"Linear SVM CV F1: {linear_cv_f1:.4f}")
print(f"RBF SVM CV F1:    {rbf_cv_f1:.4f}")

# --- Train final model on FULL training set ---
print("\n" + "="*70)
print("TRAINING FINAL MODEL ON FULL TRAINING SET")
print("="*70)

if linear_cv_f1 > rbf_cv_f1:
    print("\nBest model: Linear SVM")
    final_pipeline = pipeline_linear
    final_grid = param_grid_linear
    model_name = "Linear SVM"
else:
    print("\nBest model: RBF SVM")
    final_pipeline = pipeline_rbf
    final_grid = param_grid_rbf
    model_name = "RBF SVM"

# Final hyperparameter tuning on full training set
final_search = GridSearchCV(
    final_pipeline,
    final_grid,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring='f1_macro',
    n_jobs=-1,
    verbose=1
)

print("\nFitting final model with 5-fold CV hyperparameter tuning...")
final_search.fit(X_train_flat, y_train)

print(f"Best hyperparameters: {final_search.best_params_}")
print(f"Best CV macro F1: {final_search.best_score_:.4f}")

# --- Evaluate on held-out test set ---
print("\n" + "="*70)
print("HELD-OUT TEST SET EVALUATION")
print("="*70)

y_pred = final_search.predict(X_test_flat)

accuracy = accuracy_score(y_test, y_pred)
macro_f1 = f1_score(y_test, y_pred, average='macro')

print(f"\nTest Set Performance:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Macro F1: {macro_f1:.4f}")
print(f"\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=train_ds.classes))

# --- Confusion Matrix ---
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=train_ds.classes, 
            yticklabels=train_ds.classes)
plt.title(f'Confusion Matrix - {model_name}\nTest Accuracy: {accuracy:.4f}')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()

# --- Per-class confusion matrices ---
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.ravel()

for idx, class_name in enumerate(train_ds.classes):
    cm_class = confusion_matrix(y_test == idx, y_pred == idx)
    sns.heatmap(cm_class, annot=True, fmt='d', cmap='Greens', 
                ax=axes[idx], xticklabels=['Other', class_name], 
                yticklabels=['Other', class_name])
    axes[idx].set_title(f'{class_name}')
    axes[idx].set_ylabel('True')
    axes[idx].set_xlabel('Predicted')

plt.tight_layout()
plt.show()

# --- Summary Statistics ---
print("\n" + "="*70)
print("SUMMARY")
print("="*70)
print(f"\nNested CV estimates (unbiased performance on unseen data):")
print(f"  Linear SVM: {linear_cv_f1:.4f} ± {nested_scores_linear['test_f1_macro'].std():.4f}")
print(f"  RBF SVM:    {rbf_cv_f1:.4f} ± {nested_scores_rbf['test_f1_macro'].std():.4f}")
print(f"\nFinal test set performance ({model_name}): {macro_f1:.4f}")